# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [51]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())



In [52]:
#Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key="",
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [63]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['Clothes', 'Documentation', 'Vaccines','Doctor Phone Numbers', 'Tickets', 'Accessories', 'Insurances', 'Groceries'],
        'definition': ['List of clothes required for different climates and activities during the trip',
        'Documents needed for the trip, such as passport, visa, and travel permits',
        'Vaccination records and required vaccinations for visiting certain countries',
        'Phone numbers of my neighborhoods Doctors',
        'Details of travel tickets including flights, trains, and buses with dates and times',
        'List of accessories to bring, such as chargers, travel pillows, and toiletries',
        'Insurance policies for travel, health, or belongings in case of any incidents during the trip',
        'List of groceries for the week'] }
df = pd.DataFrame(data)
print(df)

                  table                                         definition
0               Clothes  List of clothes required for different climate...
1         Documentation  Documents needed for the trip, such as passpor...
2              Vaccines  Vaccination records and required vaccinations ...
3  Doctor Phone Numbers          Phone numbers of my neighborhoods Doctors
4               Tickets  Details of travel tickets including flights, t...
5           Accessories  List of accessories to bring, such as chargers...
6            Insurances  Insurance policies for travel, health, or belo...
7             Groceries                     List of groceries for the week


In [64]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [65]:
print(text_tables)

Clothes: List of clothes required for different climates and activities during the trip
Documentation: Documents needed for the trip, such as passport, visa, and travel permits
Vaccines: Vaccination records and required vaccinations for visiting certain countries
Doctor Phone Numbers: Phone numbers of my neighborhoods Doctors
Tickets: Details of travel tickets including flights, trains, and buses with dates and times
Accessories: List of accessories to bring, such as chargers, travel pillows, and toiletries
Insurances: Insurance policies for travel, health, or belongings in case of any incidents during the trip
Groceries: List of groceries for the week


In [66]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [67]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What would be essential to prepare for a world trip?")
print(pqt1)


Given the following tables and their content definitions,
###Tables
Clothes: List of clothes required for different climates and activities during the trip
Documentation: Documents needed for the trip, such as passport, visa, and travel permits
Vaccines: Vaccination records and required vaccinations for visiting certain countries
Doctor Phone Numbers: Phone numbers of my neighborhoods Doctors
Tickets: Details of travel tickets including flights, trains, and buses with dates and times
Accessories: List of accessories to bring, such as chargers, travel pillows, and toiletries
Insurances: Insurance policies for travel, health, or belongings in case of any incidents during the trip
Groceries: List of groceries for the week

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
What would be essential to prepare for a world trip?



In [68]:
print(return_OAI(pqt1))

```json
{
    "Tables": ["Clothes", "Documentation", "Vaccines", "Doctor Phone Numbers", "Tickets", "Accessories", "Insurances", "Groceries"]
}
```


In [69]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Now please create a SQL query that would give me a string containing the elements we need to get ready.")

In [70]:
print(return_OAI(pqt3))

```json
{
    "tables": ["Clothes", "Documentation", "Vaccines", "Accessories", "Insurances", "Groceries"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

Key Findings:

Initial Setup:

We initially provided the GPT model with several table names related to travel (e.g., "Clothes," "Documentation," "Vaccines," "Tickets," "Accessories," "Insurances") along with their descriptions.
The model did not performed well in recognizing which tables were relevant to the user’s question when asked about organizing a world trip. The returned tables did not make sense based on the context provided (e.g., returning "Clothes" and "Accessories" for packing-related queries). It returned for example the Groceries Shopping List for the Week.

We refined the prompt to broaden the scope of the query, such as asking about preparing for a year-long trip. The model still showed unessential columns.

When asked to expand the columns for each table to ensure nothing was missing, the model initially did not add any new columns or expand upon the current table structure.
A refined prompt asking for additional columns and their data types did not yield the desired result. This shows that the model sometimes struggles with creative schema generation when not explicitly guided with examples or more structured prompts.

What I Learned:

Prompt Specificity is Key: GPT performs best when prompts are detailed and structured. If the question is too vague, the model may provide incomplete or shallow responses.
Limitations of Schema Generation: While GPT can understand and work with provided tables, it struggles with independently generating new and creative additions to schemas without structured guidance.
